In [4]:
import pandas as pd

In [ ]:
dim_interno = 6.3 # cm
dim_externo = 10.5 # cm
# distancia entre as bobinas foi confirmada como aproximadamente 6.1cm (conforme o item 2)
# as bobinas estão em paralelo
# corrente está sem série


In [16]:
data = {
    "I": [0.58, 0.63, 0.71, 0.79, 0.88, 0.92, 1, 1.06, 1.11, 1.17, 1.25],
    "MHz": [30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60]

}

In [17]:
df = pd.DataFrame(data)
df

,I,MHz
0,0.58,30
1,0.63,33
2,0.71,36
3,0.79,39
4,0.88,42
5,0.92,45
6,1.00,48
7,1.06,51
8,1.11,54
9,1.17,57


In [19]:
df.to_csv('dados', sep=' ', index=False)